In [ ]:
## Creating this model: https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/BDA/Final Project')

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [ ]:
def load_dataset():
    train_dataset = pd.read_json("data/train.jsonl", lines=True)

    # train_set_x_orig = np.array(train_dataset.drop(columns = 'label')) # your train set features
    train_set_x_orig = getImages(train_dataset)
    train_set_y_orig = np.array(train_dataset['label']) # your train set labels

    test_dataset = pd.read_json("data/test_unseen.jsonl", lines=True)
    
    test_set_x_orig = getImages(test_dataset)
    # test_set_x_orig = np.array(test_dataset) # your test set features
    # test_set_y_orig = np.array(test_dataset['label']) # your test set labels

    val_dataset = pd.read_json("data/dev_unseen.jsonl", lines=True)
    
    val_set_x_orig = getImages(val_dataset)
    # val_set_x_orig = np.array(val_dataset.drop(columns = 'label')) # your train set features
    val_set_y_orig = np.array(val_dataset['label']) # your train set labels

    classes = 2 # the list of classes
    
    # train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    # val_set_y_orig = val_set_y_orig.reshape((1, val_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, val_set_x_orig, val_set_y_orig, classes

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [ ]:
import cv2

def getImages(dta):
  images = []

  for i in range(dta.shape[0]):
    r = cv2.imread("data/" + dta.loc[i, "img"])
    r.resize((224, 224, 3))
    images.append(r)
  
  return images

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, X_val_orig, Y_val_orig, classes = load_dataset()

In [ ]:
# Normalize image vectors
X_train = np.array(X_train_orig).astype('float32')/255.
X_test = np.array(X_test_orig).astype('float32')/255.
X_val = np.array(X_val_orig).astype('float32')/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 2).T
Y_val = convert_to_one_hot(Y_val_orig, 2).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))

number of training examples = 8500
number of test examples = 2000
X_train shape: (8500, 224, 224, 3)
Y_train shape: (8500, 2)
X_test shape: (2000, 224, 224, 3)


In [ ]:
np.save("x_train.npy", X_train_orig)
np.save("y_train.npy", Y_train_orig)
np.save("x_val.npy", X_val_orig)
np.save("y_val.npy", Y_val_orig)
np.save("x_test.npy", X_test_orig)

# x_train = np.float32(np.load('/content/drive/MyDrive/Deep Learning/Homework 4/fashion_mnist_train_images.npy'))
# y_train = np.int32(np.load('/content/drive/MyDrive/Deep Learning/Homework 4/fashion_mnist_train_labels.npy')).reshape(-1, 1)
# x_test  = np.float32(np.load('/content/drive/MyDrive/Deep Learning/Homework 4/fashion_mnist_test_images.npy'))
# y_test  = np.int32(np.load('/content/drive/MyDrive/Deep Learning/Homework 4/fashion_mnist_test_labels.npy')).reshape(-1, 1)

In [ ]:
img_height,img_width = 224, 224 
num_classes = 2
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape=(img_height,img_width,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs = 30, batch_size = 64)

Epoch 1/100
133/133 [==============================] - 126s 646ms/step - loss: 1.3429 - accuracy: 0.5486 - val_loss: 0.6656 - val_accuracy: 0.6296
Epoch 2/100
133/133 [==============================] - 82s 615ms/step - loss: 0.9767 - accuracy: 0.5692 - val_loss: 0.6927 - val_accuracy: 0.6296
Epoch 3/100
133/133 [==============================] - 84s 630ms/step - loss: 0.8031 - accuracy: 0.6178 - val_loss: 0.7421 - val_accuracy: 0.6296
Epoch 4/100
133/133 [==============================] - 84s 634ms/step - loss: 0.8014 - accuracy: 0.6186 - val_loss: 0.7431 - val_accuracy: 0.4870
Epoch 5/100
133/133 [==============================] - 85s 638ms/step - loss: 0.7682 - accuracy: 0.6404 - val_loss: 0.7287 - val_accuracy: 0.4926
Epoch 6/100
133/133 [==============================] - 85s 640ms/step - loss: 0.7016 - accuracy: 0.6430 - val_loss: 0.7311 - val_accuracy: 0.5278
Epoch 7/100
133/133 [==============================] - 85s 641ms/step - loss: 0.6765 - accuracy: 0.6631 - val_loss: 0.9778 

In [ ]:
model.save('model_E30.h5')

In [ ]:
preds = model.evaluate(X_val, Y_val)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

NameError: ignored